#### Persionalize 구성 흐름
user interaction 최소 조건
- At minimum 1000 interactions records from users interacting with items in your catalog.
- At minimum 25 unique user IDs with at least 2 interactions for each.

1. 데이터 탐색
2. 데이터 정제
3. S3 Bucket 생성, Policy 설정,  정제 데이터 업로드


In [3]:
import pandas as pd
import datetime
pd.set_option('display.width', 1000)

In [4]:
user_interactions = pd.read_csv('./data/user-interactions.csv')
user_interactions.head()

/tmp/ipykernel_967/1767598067.py:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  user_interactions = pd.read_csv('./data/user-interactions.csv')


,Unnamed: 0,event_time,event_type,product_id,brand,price,user_id,user_session,target,cat_0,cat_1,cat_2,cat_3
0,0,2022-11-01 00:00:14 UTC,cart,1005014,samsung,503.09,533326659,6b928be2-2bce-4640-8296-0efdf2fda22a,0,electronics,smartphone,NaN,NaN
1,1,2022-11-01 00:03:39 UTC,cart,1005115,apple,949.47,565865924,fd4bd6d4-bd14-4fdc-9aff-bd41a594f82e,0,electronics,smartphone,NaN,NaN
2,2,2022-11-01 00:05:54 UTC,cart,1002542,apple,486.80,549256216,dcbdc6e4-cd49-4ee8-95c5-e85f3c618fa1,0,electronics,smartphone,NaN,NaN
3,3,2022-11-01 00:07:22 UTC,cart,1002542,apple,486.80,549256216,dcbdc6e4-cd49-4ee8-95c5-e85f3c618fa1,0,electronics,smartphone,NaN,NaN
4,4,2022-11-01 00:10:45 UTC,cart,4804056,apple,160.57,522355747,0a1f37d1-71b7-4645-a8a7-ab91bc198a51,0,electronics,audio,headphone,NaN


In [5]:
print(user_interactions.shape)

(11495242, 13)


In [12]:
# insteraction count 확인
evnet_cnt_df = user_interactions['user_id'].value_counts().reset_index()
evnet_cnt_df.rename(columns={'index':'user_id', 'user_id':'event_cnt'},inplace=True)
evnet_cnt_df

,user_id,event_cnt
0,568782581,2134
1,549109608,1355
2,513230794,1135
3,538473314,1119
4,553431815,1050
...,...,...
2547053,616608136,1
2547054,558778912,1
2547055,514069609,1
2547056,615506166,1


In [13]:
# 두개 datafraem merge
user_interactions_joinded = pd.merge(user_interactions, evnet_cnt_df, right_on='user_id', left_on='user_id', how='inner')
user_interactions_joinded

,Unnamed: 0,event_time,event_type,product_id,brand,price,user_id,user_session,target,cat_0,cat_1,cat_2,cat_3,event_cnt
0,0,2022-11-01 00:00:14 UTC,cart,1005014,samsung,503.09,533326659,6b928be2-2bce-4640-8296-0efdf2fda22a,0,electronics,smartphone,NaN,NaN,10
1,327147,2022-11-10 21:55:30 UTC,purchase,1005174,samsung,643.23,533326659,51d80ecc-b909-4119-8228-5fe85bff4610,1,electronics,smartphone,NaN,NaN,10
2,327155,2022-11-10 21:57:23 UTC,purchase,1005174,samsung,643.23,533326659,51d80ecc-b909-4119-8228-5fe85bff4610,1,electronics,smartphone,NaN,NaN,10
3,1136178,2022-11-15 21:14:50 UTC,cart,1005174,samsung,643.23,533326659,789a5bc1-897a-4d39-af2d-11f535e87e2e,0,electronics,smartphone,NaN,NaN,10
4,1137663,2022-11-15 21:29:43 UTC,cart,1005174,samsung,643.23,533326659,789a5bc1-897a-4d39-af2d-11f535e87e2e,0,electronics,smartphone,NaN,NaN,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11495237,11495172,2023-01-31 23:36:58 UTC,purchase,2702582,dauscher,190.46,608818650,defbe193-7c47-4b70-86b3-ec14e60abc20,1,appliances,kitchen,refrigerators,NaN,1
11495238,11495191,2023-01-31 23:42:55 UTC,purchase,12704148,maxxis,49.42,608818913,83fe55ab-7c92-4b34-923f-c26973ad591a,1,NaN,NaN,NaN,NaN,1
11495239,11495207,2023-01-31 23:46:59 UTC,purchase,1003318,apple,1093.98,589879471,084c4e61-a163-4063-98f4-e45d1bb67b73,1,construction,tools,light,NaN,1
11495240,11495234,2023-01-31 23:57:33 UTC,purchase,17301779,NaN,18.16,514305364,2023c66f-f447-46cc-bb37-0788783c2c7f,1,apparel,shoes,sandals,NaN,1


In [14]:
# 무의하다고 판단되는 clolumn 1차 제거
user_interactions_joined_df = user_interactions_joinded.drop(columns=['Unnamed: 0', 'user_session', 'target'])
print(user_interactions_joined_df.shape)
user_interactions_joined_df.isnull().sum()

(11495242, 11)


event_time           0
event_type           0
product_id           0
brand           949132
price                0
user_id              0
cat_0          1527738
cat_1          1527738
cat_2          5071243
cat_3         11488757
event_cnt            0
dtype: int64

In [15]:
# Null 값이 너무 많은 column 제거
user_interactions_joined_df.drop(columns=['cat_3'], inplace=True)
user_interactions_df = user_interactions_joined_df.dropna(axis=0)
user_interactions_df.shape

(5997796, 10)

In [18]:
#  Null 값 확인
user_interactions_df.isnull().sum()
print(user_interactions_df.shape)

(5997796, 10)


In [19]:
# Dataframe 요약 정보
user_interactions_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5997796 entries, 8 to 11495241
Data columns (total 10 columns):
 #   Column      Dtype  
---  ------      -----  
 0   event_time  object 
 1   event_type  object 
 2   product_id  int64  
 3   brand       object 
 4   price       float64
 5   user_id     int64  
 6   cat_0       object 
 7   cat_1       object 
 8   cat_2       object 
 9   event_cnt   int64  
dtypes: float64(1), int64(3), object(6)
memory usage: 503.4+ MB


- event_time:  object --> Unix timestamp 타입 변환 필요

In [20]:
#  통계량 요약
user_interactions_df.describe()

,product_id,price,user_id,event_cnt
count,5.997796e+06,5.997796e+06,5.997796e+06,5.997796e+06
mean,1.013193e+07,3.377886e+02,5.552734e+08,2.599298e+01
std,2.421850e+07,3.461352e+02,3.311072e+07,7.524710e+01
min,1.000978e+06,7.700000e-01,3.948059e+07,1.000000e+00
25%,1.005003e+06,1.196900e+02,5.209589e+08,4.000000e+00
50%,1.005259e+06,2.084700e+02,5.538288e+08,9.000000e+00
75%,4.803637e+06,4.335900e+02,5.841051e+08,2.200000e+01
max,1.001444e+08,2.574070e+03,6.220901e+08,2.134000e+03


- price 값의 변동량이 너무 크므로 MinMax Scaler 필요

### evnet_time type conversion: object to Unix timestamp

In [21]:
user_interactions_df2 = user_interactions_df.copy()

# 유효하지 않는  format은  NaT으로 변환
user_interactions_df2['event_time'] = pd.to_datetime(user_interactions_df2['event_time'], format="%Y-%m-%d %H:%M:%S UTC", errors='coerce')
user_interactions_df2.dropna()

,event_time,event_type,product_id,brand,price,user_id,cat_0,cat_1,cat_2,event_cnt
8,2022-12-16 03:15:54,cart,17300764,lanvin,45.71,533326659,apparel,shoes,sandals,10
9,2022-12-25 16:32:01,cart,17300814,lacoste,25.78,533326659,apparel,shoes,sandals,10
18,2022-11-01 00:10:45,cart,4804056,apple,160.57,522355747,electronics,audio,headphone,8
21,2023-02-10 18:16:24,cart,100086235,samsung,114.89,522355747,construction,tools,light,8
22,2023-02-10 18:17:37,cart,100086203,samsung,114.89,522355747,construction,tools,light,8
...,...,...,...,...,...,...,...,...,...,...
11495232,2023-01-31 23:18:14,purchase,1002544,apple,410.42,608815627,construction,tools,light,1
11495233,2023-01-31 23:27:19,purchase,1002547,apple,411.57,608817284,construction,tools,light,1
11495237,2023-01-31 23:36:58,purchase,2702582,dauscher,190.46,608818650,appliances,kitchen,refrigerators,1
11495239,2023-01-31 23:46:59,purchase,1003318,apple,1093.98,589879471,construction,tools,light,1


In [22]:

# event_time의 datetime을 Unix timestamp 타입으로
user_interactions_df2['event_time'] = user_interactions_df2.event_time.apply(lambda x : (x-datetime.datetime(1970,1,1)).total_seconds())
user_interactions_df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5997796 entries, 8 to 11495241
Data columns (total 10 columns):
 #   Column      Dtype  
---  ------      -----  
 0   event_time  float64
 1   event_type  object 
 2   product_id  int64  
 3   brand       object 
 4   price       float64
 5   user_id     int64  
 6   cat_0       object 
 7   cat_1       object 
 8   cat_2       object 
 9   event_cnt   int64  
dtypes: float64(2), int64(3), object(5)
memory usage: 503.4+ MB


In [23]:
# price MinMax Sacler 적용
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
user_interactions_df2['price'] = scaler.fit_transform(user_interactions_df2[['price']])

In [24]:
user_interactions_df2.head()

,event_time,event_type,product_id,brand,price,user_id,cat_0,cat_1,cat_2,event_cnt
8,1.671161e+09,cart,17300764,lanvin,0.017464,533326659,apparel,shoes,sandals,10
9,1.671986e+09,cart,17300814,lacoste,0.009719,533326659,apparel,shoes,sandals,10
18,1.667261e+09,cart,4804056,apple,0.062099,522355747,electronics,audio,headphone,8
21,1.676053e+09,cart,100086235,samsung,0.044348,522355747,construction,tools,light,8
22,1.676053e+09,cart,100086203,samsung,0.044348,522355747,construction,tools,light,8


In [25]:
# rename columne:  Personalize를 위한 필수 항목: USER_ID, ITEM_ID, TIMESTAMP
user_interactions_df2.rename(columns={"user_id": "USER_ID", "product_id": "ITEM_ID", "event_time": "TIMESTAMP", "event_type": "EVENT_TYPE", "brand": "BRAND",
                                      "price": "PRICE", "cat_0": "CAT_0", "cat_1": "CAT_1", "cat_2": "CAT_2", 'event_cnt':'EVENT_CNT' }, inplace=True)
user_interactions_df2 = user_interactions_df2.reindex(columns=['USER_ID', 'ITEM_ID', 'TIMESTAMP', "EVENT_TYPE", "PRICE", "BRAND", "CAT_0", "CAT_1", "CAT_2", 'EVENT_CNT'])

In [26]:
user_interactions_df2.head()

,USER_ID,ITEM_ID,TIMESTAMP,EVENT_TYPE,PRICE,BRAND,CAT_0,CAT_1,CAT_2,EVENT_CNT
8,533326659,17300764,1.671161e+09,cart,0.017464,lanvin,apparel,shoes,sandals,10
9,533326659,17300814,1.671986e+09,cart,0.009719,lacoste,apparel,shoes,sandals,10
18,522355747,4804056,1.667261e+09,cart,0.062099,apple,electronics,audio,headphone,8
21,522355747,100086235,1.676053e+09,cart,0.044348,samsung,construction,tools,light,8
22,522355747,100086203,1.676053e+09,cart,0.044348,samsung,construction,tools,light,8


In [27]:
# 정제된 데이터를 파일로 저장
user_interactions_df2.to_csv("./data/cleaned_user_interactions.csv", index=False)